In [ ]:
!pip install https://github.com/scikit-learn-contrib/scikit-learn-extra/archive/master.zip

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import timedelta
from sklearn.cluster import KMeans
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import PCA
import plotly.figure_factory as ff


cve = pd.read_csv('../input/cve-common-vulnerabilities-and-exposures/cve.csv', header=0, index_col=0)
products = pd.read_csv('../input/cve-common-vulnerabilities-and-exposures/products.csv', header=0, index_col=0)
vendors = pd.read_csv('../input/cve-common-vulnerabilities-and-exposures/vendors.csv', header=0, index_col=0)
vendor_product = pd.read_csv('../input/cve-common-vulnerabilities-and-exposures/vendor_product.csv', header=0, index_col=0)

cve.pub_date = pd.to_datetime(cve.pub_date)

# 0. Exploit Types (CWEs) and Products

First, we'll explore the relationship between exploits and products. We'll focus on how to cluster those products according to the exploits that affect them. Each product will be represented as a vector of exploit types, normalized row-wise.

In [ ]:
K = 500
M = 100
L = 10
thresh = .01

cwes = pd.get_dummies(cve.cwe_name)
top_K = products.vulnerable_product.value_counts()[:K]
counts = top_K.values

X = products[products.vulnerable_product.isin(
    top_K.index
    )].join(
        cwes
    ).groupby(
        'vulnerable_product'
).sum()

# Normalize
X = X.divide(X.sum(axis=1), axis=0)

# Sparsify (to prevent big products from being clustered together by virtue of being big)
below_threshold = X < thresh
X[below_threshold] = 0

Z = KernelPCA(n_components=2, kernel="cosine").fit_transform(X)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=Z[:,0][L:M],
        y=Z[:,1][L:M],
        mode="markers",
        text=top_K.index[L:M],
        marker=dict(
            size=2 * np.sqrt(counts[L:M]),
            color='#bbbbbb',
            showscale=False
        )
    )
)
fig.add_trace(
    go.Scatter(
        x=Z[:,0][:L],
        y=Z[:,1][:L],
        mode="markers+text",
        text=np.vectorize(lambda x: " ".join(map(lambda x: x.title() if len(x) > 2 else x.upper(), x.split("_"))))(top_K.index[:L]),
        textposition = "top center",
        marker=dict(
            size=2 * np.sqrt(counts[:L]),
            color='#93a1a1',
            showscale=False,
        )
    )
)

fig.update_layout(
    height=800,
    showlegend=False,
    title=dict(
        text="Plotting products using exploit types",
        xref="paper",
        x=0., y=1.
    ),
    font=dict(
        family="Arial",
        size=14,
        color="#586e75"
    ),
    xaxis=dict(
        showgrid=False,
        showticklabels=False
    ),
    yaxis=dict(
        showgrid=False,
        showticklabels=False
    ),
    annotations=[
        dict(
            xref='paper',
            yref='paper',
            x=0., y=1.1,
            showarrow=False,
            text ="All products are projected into two dimensions using KernelPCA with cosine similarity as the chosen kernel.<br>" + 
            "Products are sized proportionally to the how many CVE entries affect a given product. The top 10 products<br>are labelled, the top 100 are projected.",
            valign='top',
            align='left'
        ),
    ],
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    bargap=0
)

In [ ]:
fig = go.Figure()

solarized_colors = ['#dc322f', '#6c71c4', '#268bd2', '#2aa198', '#073642']
V = vendors.vendor.value_counts()[5:0:-1].index
top_K_vp = top_K.reset_index().join(vendor_product.set_index('product'), on='index')

ndxs = top_K_vp[~top_K_vp.vendor.isin(V)].index
fig.add_trace(
    go.Scatter(
        x=Z[:,0][ndxs],
        y=Z[:,1][ndxs],
        mode="markers",
        name='others',
        text=np.vectorize(lambda x: " ".join(map(lambda x: x.title() if len(x) > 2 else x.upper(), x.split("_"))))(top_K.index[ndxs]),
        marker=dict(
            size=2 * np.sqrt(counts[ndxs]),
            color='#dddddd',
            showscale=False,
        )
    )
)

for ndx, vendor in enumerate(V):
    ndxs = top_K_vp[top_K_vp.vendor == vendor].index
    
    fig.add_trace(
        go.Scatter(
            x=Z[:,0][ndxs],
            y=Z[:,1][ndxs],
            mode="markers",
            name=vendor,
            text=np.vectorize(lambda x: " ".join(map(lambda x: x.title() if len(x) > 2 else x.upper(), x.split("_"))))(top_K.index[ndxs]),
            marker=dict(
                size=2 * np.sqrt(counts[ndxs]),
                color=solarized_colors[ndx],
                showscale=False,
            )
        )
    )
    
fig.update_layout(
    legend=dict(x=0., y=1.),
    height=800,
    title=dict(
        text="Visualizing vendors",
        xref="paper",
        x=0., y=1.
    ),
    font=dict(
        family="Arial",
        size=14,
        color="#586e75"
    ),
    xaxis=dict(
        showgrid=False,
        showticklabels=False
    ),
    yaxis=dict(
        showgrid=False,
        showticklabels=False
    ),
    annotations=[
        dict(
            xref='paper',
            yref='paper',
            x=0., y=1.1,
            showarrow=False,
            text ="Vendors are one inherent clustering already present in the data. Here's how KernelPCA with<br>cosine similarity distributes products from the top 5 vendors.",
            valign='top',
            align='left'
        ),
    ],
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)
    
fig.show()

In [ ]:
from sklearn_extra.cluster import KMedoids

colors = KMedoids(n_clusters=3, metric='cosine', random_state=0).fit_predict(X[:K])
fig = go.Figure()
L = 10


fig.add_trace(
     go.Scatter(
        x=Z[:,0][L:K],
        y=Z[:,1][L:K],
        text=np.vectorize(lambda x: " ".join(map(lambda x: x.title() if len(x) > 2 else x.upper(), x.split("_"))))(top_K.index[L:K]),
        marker=dict(
            size=2 * np.sqrt(counts[L:K]), # Make the area proportional
            color=colors[L:K],
            colorscale = [[0,'rgba(220,50,47,.2)'],[.5, 'rgba(108,113,196,.2)'],[1, 'rgba(38,139,210,.2)']]
        ),
        mode='markers'
    )
)

fig.add_trace(go.Scatter(
        x=Z[:,0][:L],
        y=Z[:,1][:L],
        text=np.vectorize(lambda x: " ".join(map(lambda x: x.title() if len(x) > 2 else x.upper(), x.split("_"))))(top_K.index[:L]),
        textposition='top center',
        marker=dict(
            size=2 * np.sqrt(counts[:L]), # Make the area proportional
            color=colors[:L],
            colorscale = [[0,'rgb(220,50,47)'],[.5, 'rgb(108,113,196)'],[1, 'rgb(38,139,210)']]
        ),
        mode='markers+text'
    )
)

fig.update_layout(
    showlegend=False,
    height=800,
    title=dict(
        text="2-D Clustering, K-Medoids",
        xref="paper",
        x=0., y=1.
    ),
    font=dict(
        family="Arial",
        size=14,
        color="#586e75"
    ),
    xaxis=dict(
        showgrid=False,
        showticklabels=False
    ),
    yaxis=dict(
        showgrid=False,
        showticklabels=False
    ),
    annotations=[
        dict(
            xref='paper',
            yref='paper',
            x=0., y=1.1,
            showarrow=False,
            text ="Though the data has significant overlap, it appears loosely ternary. Therefore, here is the" +
            " 3-Medoids clustering<br>with cosine similarity as the similarity metric",
            valign='top',
            align='left'
        ),
    ],
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()

In [ ]:
H = 50
top_H = top_K = products.vulnerable_product.value_counts()[:H]
fig = ff.create_dendrogram(
    X[X.index.isin(top_H.index)].sort_index(),
    labels=np.vectorize(lambda x: " ".join(map(lambda x: x.title() if len(x) > 2 else x.upper(), x.split("_"))))(top_H.sort_index().index),
    orientation='left')

fig.update_layout(
    height=1000,
    showlegend=False,
    title=dict(
        text="An intuitive signal, Hierarchical Agglomerative Clustering",
        xref="paper",
        x=0., y=1.
    ),
    font=dict(
        family="Arial",
        size=14,
        color="#586e75"
    ),
    xaxis=dict(
        showgrid=False,
        showticklabels=False,
        ticklen=0
    ),
    yaxis=dict(
        showgrid=False,
        ticklen=0
    ),
    annotations=[
        dict(
            xref='paper',
            yref='paper',
            x=0., y=1.1,
            showarrow=False,
            text="In a complete linkage heirarchical clustering, products from the same product line<br>" +
            "or vendor cluster nicely. Examples include the Windows family (top) or Debian<br>" +
            "and Ubuntu (top-middle). The top 50 products are shown.",
            valign='top',
            align='left'
        ),
    ],
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()